# Loading data

In [1]:
import pandas as pd
import numpy as np

In [2]:
myDf=pd.read_csv("data/test.csv")

# Preprocessing

In [3]:
myDf["text"]=myDf["text"].apply(lambda x:x+" <end>")
myDf["tag"]=myDf["tag"].apply(lambda x:x+" END")
myDf[:1]

,text,tag
0,将 军 百 战 死 <end>,B I B I S END


In [4]:
myDf.dropna(inplace=True)

# Transforming data to one-hot embedding to generalize X

In [5]:
wordIndexDict={"<pad>":0}
wi=1
for row in myDf["text"].values.tolist():
    if type(row)==float:
        print(row)
        break
    for word in row.split(" "):
        if word not in wordIndexDict:
            wordIndexDict[word]=wi
            wi+=1
vocabSize=wi

In [6]:
maxLen=max(len(row) for row in myDf["text"].values.tolist())
sequenceLengths=[len(row) for row in myDf["text"].values.tolist()]

In [7]:
myDf["text"]=myDf["text"].apply(lambda x:[wordIndexDict[word] for word in x.split()])

In [8]:
import tensorflow as tf
X=tf.keras.preprocessing.sequence.pad_sequences(myDf["text"],
                                                value=wordIndexDict["<pad>"],
                                                padding='post',
                                                maxlen=maxLen)
X

array([[ 1,  2,  3,  4,  5,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0],
       [ 1,  2,  7,  8,  9,  4, 10,  6,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0],
       [ 1,  1,  2,  8,  9,  4, 11,  6,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0],
       [ 2,  4,  1,  2,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0],
       [ 1,  2,  4,  1,  2,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0],
       [ 1,  2,  4,  1,  2,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0]])

# Generalizing Y

In [9]:
import tqdm
import re

myDf["tag"]=myDf["tag"].apply(lambda x:re.sub("\-\S+","",x))

tagIndexDict = {"PAD": 0}
ti = 1
for row in tqdm.tqdm(myDf["tag"].values.tolist()):
    for tag in row.split(" "):
        if tag not in tagIndexDict:
            tagIndexDict[tag] = ti
            ti += 1
tagSum = len(list(tagIndexDict.keys()))
myDf["tag"] = myDf["tag"].apply(lambda x:x.split()+["PAD" for i in range(maxLen-len(x.split()))])
myDf["tag"] = myDf["tag"].apply(lambda x:[tagIndexDict[tagItem] for tagItem in x])
# myDf["tag"] = myDf["tag"].apply(lambda x: [[0 if tagI != tagIndexDict[tagItem] else 1
#                                             for tagI in range(len(tagIndexDict))]
#                                             for tagItem in x])
y=np.array(myDf["tag"].values.tolist())

100%|██████████| 6/6 [00:00<?, ?it/s]


In [10]:
y.shape # it is OK whether y is one-hot embedding or not

(6, 19)

# Generalizing Model

In [11]:
from BiLSTMCRF import MyBiLSTMCRF
myModel=MyBiLSTMCRF(vocabSize,maxLen, tagIndexDict,tagSum,sequenceLengths)

In [12]:
myModel.myBiLSTMCRF.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 19, 100)           1200      
_________________________________________________________________
bidirectional (Bidirectional (None, 19, 5)             4240      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 19, 5)             440       
_________________________________________________________________
crf_layer (CRF)              (None, 19)                65        
Total params: 5,945
Trainable params: 5,945
Non-trainable params: 0
_________________________________________________________________


# training model

In [13]:
history=myModel.fit(X,y,epochs=1500)

oss: 0.2603
Epoch 1260/1500
1/1 [==============================] - 0s 13ms/step - loss: 0.2602
Epoch 1261/1500
1/1 [==============================] - 0s 14ms/step - loss: 0.2602
Epoch 1262/1500
1/1 [==============================] - 0s 15ms/step - loss: 0.2601
Epoch 1263/1500
1/1 [==============================] - 0s 14ms/step - loss: 0.2601
Epoch 1264/1500
1/1 [==============================] - 0s 15ms/step - loss: 0.2601
Epoch 1265/1500
1/1 [==============================] - 0s 11ms/step - loss: 0.2600
Epoch 1266/1500
1/1 [==============================] - 0s 13ms/step - loss: 0.2600
Epoch 1267/1500
1/1 [==============================] - 0s 14ms/step - loss: 0.2600
Epoch 1268/1500
1/1 [==============================] - 0s 13ms/step - loss: 0.2599
Epoch 1269/1500
1/1 [==============================] - 0s 16ms/step - loss: 0.2599
Epoch 1270/1500
1/1 [==============================] - 0s 13ms/step - loss: 0.2598
Epoch 1271/1500
1/1 [==============================] - 0s 15ms/step - loss:

# predicting

In [14]:
testI=2

In [15]:
preY=myModel.predict(X)[testI]

In [16]:
indexTagDict=dict(list(zip(list(tagIndexDict.values()),list(tagIndexDict.keys()))))
indexWordDict=dict(list(zip(list(wordIndexDict.values()),list(wordIndexDict.keys()))))

sentenceList=[indexWordDict[wordItem] for wordItem in X[testI]]
sentenceList=sentenceList[:sentenceList.index("<end>")]

tagList=[indexTagDict[tagItem] for tagItem in preY]
tagList=tagList[:tagList.index("END")]

print(" ".join(sentenceList))
print(" ".join(tagList))

将 将 军 带 上 战 车
S B I B I B I
